In [ ]:
!pip install PyMuPDF
!pip install Pix2Text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.6/224.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 711.2/711.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import fitz  # PyMuPDF
from pix2text import Pix2Text
import google.generativeai as genai

## Convert PDFs to Markdowns

In [ ]:
# code from pixImple.py

def run_pix2text(img_fp, page_number):
    p2t = Pix2Text.from_config(device="cpu")
    kwargs = {
        "resized_shape": 1280,
        "mfr_batch_size": 1,
        "embed_sep": (" $", "$ "),
        "isolated_sep": ("$$", "$$"),
        "line_sep": "\n",
        "auto_line_break": True,
        "det_text_bbox_max_width_expand_ratio": 0.3,
        "det_text_bbox_max_height_expand_ratio": 0.2,
        "embed_ratio_threshold": 0.6,
        "table_as_image": True,
        "formula_rec_kwargs": {}
    }
    try:
        page = p2t.recognize_page(img_fp, page_number=page_number, **kwargs)
        return page
    except Exception as e:
        print(f"Error processing page {page_number}: {e}")
        return None

def save_pix2text_output(page, page_number, output_page_dir):
    os.makedirs(output_page_dir, exist_ok=True)
    output_md_path = os.path.join(output_page_dir, f'page_{page_number}.md')
    try:
        page.to_markdown(output_md_path)
        print(f"Saved Markdown to {output_md_path}")
    except Exception as e:
        print(f"Error saving Markdown for page {page_number}: {e}")

    if hasattr(page, 'images') and page.images:
        for img_name, img_data in page.images.items():
            output_img_path = os.path.join(output_page_dir, f'{page_number}_img_{img_name}')
            try:
                with open(output_img_path, 'wb') as img_file:
                    img_file.write(img_data)
                print(f"Saved image to {output_img_path}")
            except Exception as e:
                print(f"Error saving image {img_name} for page {page_number}: {e}")

# modified from pixImple to process PDFs by each page into markdown files
def process_pdfs(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    pdf_files = [f for f in os.listdir(input_dir) if f.endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_output_dir = os.path.join(output_dir, os.path.splitext(pdf_file)[0])
        os.makedirs(pdf_output_dir, exist_ok=True)

        doc = fitz.open(pdf_path)
        for page_number in range(len(doc)):
            page = doc.load_page(page_number)
            img_fp = os.path.join(pdf_output_dir, f'page_{page_number + 1}.png')
            pix = page.get_pixmap()
            pix.save(img_fp)

            result = run_pix2text(img_fp, page_number + 1)
            if result:
                save_pix2text_output(result, page_number + 1, pdf_output_dir)

        doc.close()
        print(f"Completed processing {pdf_file}")

# Process pdfs into output folder
input_dir = 'input'
output_dir = 'output'
process_pdfs(input_dir, output_dir)



0: 1024x672 3 titles, 8 plain texts, 2 abandons, 4007.0ms
Speed: 6.9ms preprocess, 4007.0ms inference, 1.2ms postprocess per image at shape (1, 3, 1024, 672)
Loading /root/.pix2text/1.1/mfd-onnx/mfd-v20240618.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

WARNING ⚠️ imgsz=[258, 1280] must be multiple of max stride 32, updating to [288, 1280]
0: 288x1280 (no detections), 422.8ms
Speed: 4.2ms preprocess, 422.8ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 1280)


0it [00:00, ?it/s]



WARNING ⚠️ imgsz=[506, 1280] must be multiple of max stride 32, updating to [512, 1280]
0: 512x1280 (no detections), 1142.8ms
Speed: 6.2ms preprocess, 1142.8ms inference, 0.9ms postprocess per image at shape (1, 3, 512, 1280)


0it [00:00, ?it/s]



WARNING ⚠️ imgsz=[314, 1280] must be multiple of max stride 32, updating to [320, 1280]
0: 320x1280 (no detections), 448.9ms
Speed: 4.3ms preprocess, 448.9ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 1280)


...

Saved Markdown to output/2073/page_8.md

0: 1024x800 1 plain text, 5972.3ms
Speed: 7.6ms preprocess, 5972.3ms inference, 1.8ms postprocess per image at shape (1, 3, 1024, 800)
Loading /root/.pix2text/1.1/mfd-onnx/mfd-v20240618.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

WARNING ⚠️ imgsz=[694, 1280] must be multiple of max stride 32, updating to [704, 1280]
0: 704x1280 3 embeddings, 1481.9ms
Speed: 12.2ms preprocess, 1481.9ms inference, 1.4ms postprocess per image at shape (1, 3, 704, 1280)


100%|██████████| 3/3 [00:03<00:00,  1.02s/it]



WARNING ⚠️ imgsz=[1656, 1280] must be multiple of max stride 32, updating to [1664, 1280]
0: 1664x1280 (no detections), 3415.2ms
Speed: 17.6ms preprocess, 3415.2ms inference, 1.7ms postprocess per image at shape (1, 3, 1664, 1280)


0it [00:00, ?it/s]


Saved Markdown to output/2073/page_9.md
Completed processing 2073.pdf


## Combine Markdowns into One

In [ ]:
# function to combine all markdowns within each pdf folder to output single markdown
def combine_output_markdowns(output_dir):
    for folder in os.listdir(output_dir):
        folder_path = os.path.join(output_dir, folder)
        if os.path.isdir(folder_path):
            combined_md_path = os.path.join(folder_path, f'{folder}_combined.md')
            with open(combined_md_path, 'w', encoding='utf-8') as combined_file:
                markdown_found = False
                # Traverse subdirectories to find output.md
                for subdir in os.listdir(folder_path):
                    subdir_path = os.path.join(folder_path, subdir)
                    if os.path.isdir(subdir_path):
                        output_md_path = os.path.join(subdir_path, 'output.md')
                        if os.path.isfile(output_md_path):
                            with open(output_md_path, 'r', encoding='utf-8') as md_file:
                                content = md_file.read().strip()
                                if content:
                                    combined_file.write(f'# {subdir}\n\n')  # Subdir name as header
                                    combined_file.write(content + '\n\n')
                                    markdown_found = True

                if not markdown_found:
                    print(f"No output.md files found in {folder_path}, combined file will be empty.")
                else:
                    print(f"Combined markdown saved to {combined_md_path}")


In [ ]:
# Combine markdowns
output_dir = 'output'
combine_output_markdowns(output_dir)

Combined markdown saved to output/2099/2099_combined.md
Combined markdown saved to output/2073/2073_combined.md


## Build JSON from Combined Markdown

In [ ]:
# Set your API key
genai.configure(api_key="-")
# Load the JSON schema
with open("/content/json_schema.txt", "r") as file:
    json_schema = file.read()

# Function to process a single Markdown file
def process_markdown_file(md_file_path):
    with open(md_file_path, "r") as file:
        markdown_content = file.read()

    # Prepare the prompt
    prompt = f"""
    Based on the following JSON schema, extract the information from the provided Markdown content and generate a JSON output:

    JSON Schema:
    {json_schema}

    Markdown Content:
    {markdown_content}

    Please produce a JSON output following the schema exactly.
    """

    # Call the Gemini API
    # model = genai.GenerativeModel('gemini-2.0-pro-exp') #or gemini-pro-vision if you need it.
    model = genai.GenerativeModel('gemini-2.0-flash') #or gemini-pro-vision if you need it.
    response = model.generate_content(prompt)

    # Get the generated JSON
    generated_json = response.text

    # Save JSON file in the same folder as the Markdown file
    output_file_path = os.path.splitext(md_file_path)[0] + ".json"
    with open(output_file_path, "w") as file:
        file.write(generated_json)

    print(f"JSON output saved to {output_file_path}")


In [ ]:
# Traverse multiple folders and process each Markdown file
root_dir = "output"
target_subdirs = ["2073", "2099"]

for subdir in target_subdirs:
    subdir_path = os.path.join(root_dir, subdir)

    for file in os.listdir(subdir_path):
        if file.endswith(".md") and os.path.isfile(os.path.join(subdir_path, file)):
            md_file_path = os.path.join(subdir_path, file)

            # Check if the .md file is directly in the target subdirectory (not in a nested subdirectory)
            if os.path.dirname(md_file_path) == subdir_path:
                print(f"Processing {md_file_path}...")
                process_markdown_file(md_file_path)

print("Processing complete.")

Processing output/2073/2073_combined.md...
JSON output saved to output/2073/2073_combined.json
Processing output/2099/2099_combined.md...
JSON output saved to output/2099/2099_combined.json
Processing complete.
